In [1]:
import pandas as pd

In [2]:
pwd

'd:\\OneDrive - CGIAR\\agrilac\\icc'

In [ ]:
prec = pd.read_excel("D:\\OneDrive - CGIAR\\agrilac\\icc\\cengicana_prec.xlsx")

prec.info()

NameError: name 'pd' is not defined

In [4]:
pac = pd.read_excel('oni_pacifico.xlsx')
atl =pd.read_excel('oni_atlantico.xlsx')

In [5]:
pac.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 312 entries, 0 to 311
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   year    312 non-null    int64  
 1   month   312 non-null    int64  
 2   oni     312 non-null    float64
dtypes: float64(1), int64(2)
memory usage: 7.4 KB


In [7]:
import pandas as pd
import numpy as np
from sklearn.metrics import mean_absolute_error, mean_squared_error
import warnings

def maape(y_true, y_pred):
    with np.errstate(divide='ignore', invalid='ignore'):
        arctan = np.arctan(np.abs((y_true - y_pred) / (y_true + 1e-10)))
    return np.mean(arctan)

def validate_input(df, required_columns, df_name):
    if not isinstance(df, pd.DataFrame):
        raise ValueError(f"{df_name} debe ser un DataFrame de pandas.")
    missing_cols = [col for col in required_columns if col not in df.columns]
    if missing_cols:
        raise ValueError(f"{df_name} falta columnas requeridas: {missing_cols}")

def build_series(df, year, month, window=6):
    df_sorted = df.sort_values(['year', 'month']).reset_index(drop=True)
    series = []
    current_year, current_month = year, month

    for _ in range(window):
        match = df_sorted[(df_sorted['year'] == current_year) & (df_sorted['month'] == current_month)]
        if match.empty:
            return None  # No se puede construir la serie completa
        series.append(match.iloc[0])

        # Retroceder un mes
        if current_month == 1:
            current_month = 12
            current_year -= 1
        else:
            current_month -= 1

    series_df = pd.DataFrame(series[::-1])  # Revertir para mantener el orden cronológico
    return series_df.reset_index(drop=True)

def build_past_series(df, year, month, window=6):
    df_sorted = df.sort_values(['year', 'month']).reset_index(drop=True)
    years = df_sorted['year'].unique()
    past_years = years[years < year]
    series_list = []

    for past_year in past_years:
        series = build_series(df_sorted, past_year, month, window)
        if series is not None:
            series_list.append((past_year, series))

    return series_list

def compare_series(base_series, past_series_list, var, base_name=None):
    results = []
    y_true = base_series[var].values

    for year, past_series in past_series_list:
        y_pred = past_series[var].values

        if len(y_true) != len(y_pred):
            warnings.warn(f"El tamaño de la serie base y {year} no coincide. Se omite este año.")
            continue

        if not all(base_series['month'].values == past_series['month'].values):
            warnings.warn(f"Los meses de la serie base y {year} no coinciden exactamente. Se omite este año.")
            continue

        result = {
            'base': base_name if base_name else var,  # <<< Aquí corregimos la base explícita
            'año_comparado': year,
            'MAAPE': maape(y_true, y_pred),
            'RMSE': mean_squared_error(y_true, y_pred, squared=False),
            'MAE': mean_absolute_error(y_true, y_pred)
        }
        results.append(result)

    return results

def comparar_dataframes(prec_df, atl_df, pac_df, year, month):
    validate_input(prec_df, ['year', 'month', 'prec'], 'prec_df')
    validate_input(atl_df, ['year', 'month', 'oni'], 'atl_df')
    validate_input(pac_df, ['year', 'month', 'oni'], 'pac_df')

    # Construir series base
    prec_base = build_series(prec_df, year, month)
    atl_base = build_series(atl_df, year, month)
    pac_base = build_series(pac_df, year, month)

    if prec_base is None or atl_base is None or pac_base is None:
        raise ValueError(f"No se puede construir la serie completa para el año {year} y mes {month}. Verifique los datos.")

    # Construir series de años pasados
    prec_past = build_past_series(prec_df, year, month)
    atl_past = build_past_series(atl_df, year, month)
    pac_past = build_past_series(pac_df, year, month)

    # Comparaciones (ahora indicando base explícita)
    prec_results = compare_series(prec_base, prec_past, 'prec', base_name='prec')
    atl_results = compare_series(atl_base, atl_past, 'oni', base_name='atl')
    pac_results = compare_series(pac_base, pac_past, 'oni', base_name='pac')

    all_results = prec_results + atl_results + pac_results
    results_df = pd.DataFrame(all_results)

    return results_df#.sort_values(['base', 'MAAPE']).reset_index(drop=True)

# Ejemplo de uso:
# resultados = comparar_dataframes(prec, atl, pac, 2020, 4)
# print(resultados)


In [8]:
def ponderar_resultados(resultados_df):
    # Definir pesos
    pesos = {
        'prec': 0.4,
        'pac': 0.35,
        'atl': 0.25
    }
    
    # Validar que todas las bases estén presentes
    bases_validas = ['prec', 'pac', 'atl']
    presentes = resultados_df['base'].unique()
    for base in bases_validas:
        if base not in presentes:
            warnings.warn(f"La base '{base}' no está presente en los datos.")
    
    combinados = []

    # Obtener todos los años únicos
    años = resultados_df['año_comparado'].unique()

    for año in sorted(años):
        datos_año = resultados_df[resultados_df['año_comparado'] == año]

        for metrica in ['MAAPE', 'RMSE', 'MAE']:
            valor = 0
            for _, fila in datos_año.iterrows():
                base = fila['base']
                if base in pesos:
                    valor += fila[metrica] * pesos[base]
            combinados.append({
                'año_comparado': año,
                'metrica': metrica,
                'valor_ponderado': valor
            })

    combinados_df = pd.DataFrame(combinados)
    return combinados_df

# Ejemplo de uso:
# ponderados_df = ponderar_resultados(resultados)
# print(ponderados_df)


In [16]:
year=2021

In [17]:
resultados = comparar_dataframes(prec, atl, pac, year, 4)

In [18]:
ponderados_df = ponderar_resultados(resultados)
ponderados_df[ponderados_df.metrica == 'RMSE'].sort_values('valor_ponderado').head(5)

,año_comparado,metrica,valor_ponderado
58,2018,RMSE,11.955635
37,2011,RMSE,14.194651
13,2003,RMSE,15.982625
40,2012,RMSE,23.769844
64,2020,RMSE,27.719166


In [19]:
real = pd.read_excel('week_prec_cengicana.xlsx')
real.head(2)

,year,sem_iell,fell
0,2025,NaN,NaN
1,2024,23.0,66.0


In [20]:
mape =ponderados_df[ponderados_df.metrica == 'RMSE'].sort_values('valor_ponderado').merge(real,how='inner', left_on='año_comparado', right_on='year')
print(mape.head())
mape.sem_iell.head().mean()

   año_comparado metrica  valor_ponderado  year  sem_iell  fell
0           2018    RMSE        11.955635  2018      18.0  63.0
1           2011    RMSE        14.194651  2011      19.0  60.0
2           2003    RMSE        15.982625  2003      24.0  65.0
3           2012    RMSE        23.769844  2012      16.0  68.0
4           2020    RMSE        27.719166  2020      24.0  68.0


20.2

In [21]:
resultados_ = comparar_dataframes(prec, atl, pac, year, 3)
ponderados_df_ = ponderar_resultados(resultados_)
ponderados_df_[ponderados_df_.metrica == 'RMSE'].sort_values('valor_ponderado').head()

,año_comparado,metrica,valor_ponderado
13,2003,RMSE,21.052579
64,2020,RMSE,23.481814
43,2013,RMSE,31.406747
40,2012,RMSE,33.912385
10,2002,RMSE,34.543417


In [22]:
mape_ =ponderados_df_[ponderados_df_.metrica == 'RMSE'].sort_values('valor_ponderado').merge(real,how='inner', left_on='año_comparado', right_on='year')
print(mape_.head())
mape_.sem_iell.head().mean()

   año_comparado metrica  valor_ponderado  year  sem_iell  fell
0           2003    RMSE        21.052579  2003      24.0  65.0
1           2020    RMSE        23.481814  2020      24.0  68.0
2           2013    RMSE        31.406747  2013      23.0  66.0
3           2012    RMSE        33.912385  2012      16.0  68.0
4           2002    RMSE        34.543417  2002      23.0  67.0


22.0